# Tarea 1

En esta tarea vas a programar tu mismo el algoritmo de descenso del gradiente, en dos versiones, y comparar ambos resultados. Para probar tu modelo vas a tener que usar el _dataset_ de gustos de helados que vimos en clases.

## Parte 1 (3 pts): gradient descent simple

En el codigo de abajo hay una clase llamada `LogisticRegression` cuyo constructor recibe como parámetro el número de _features_ que espera recibir. Tienes que completar esta clase para que pueda entrenar y predecir. Lo que necesitas es:

- Programar el método `train`, que vendría a ser equivalente al método `fit` de Scikit Learn. Tienes que utilizar el algoritmo _Gradient Descent_ visto en clases.
- Programar el método `predict` que asume que tu modelo ya está entrenado.

Para hacer esto puedes hacer los supuestos razonables que estimes conveniente. Además, si te acomoda trabajar sin clases puedes hacerlo, mientras uses el algoritmo de _Gradient Descent_. **Importante**: puedes asumir que una instancia es "positiva" (es decir, pertenece a la clase 1) si la probabilidad calculada es mayor o igual a 0.5.

Recuerda además que el gradiente de la función objetivo para cada $\beta_i$ es:

$$
\frac{\delta}{\delta \beta_i}L(\beta) = \frac{1}{m} \sum_{1 \leq j \leq m} (\sigma(\beta^T x^j) - y_j) x_i^j
$$

Donde $L(\beta)$ es la función de verosimilitud, $\beta$ es el vector de coeficientes para la regresión, tenemos $m$ filas en nuestro _dataset_, $\sigma(x)$ es la función $\frac{1}{1 + e^{-x}}$, $x^j$ es la fila $j$ de nuestro dataset (y asociado tiene su respuesta $y_j$) y finalmente $x_i^j$ es la columna $i$ de la fila $j$ en nuestro _dataset_.

In [122]:
# Tienes que programar la parte 1 aquí
import numpy as np
import matplotlib.pyplot as plt
# La función sigmoide
def sigmoid(x):    
    output = 1 / (1 + np.exp(-x))
    return output


class LogisticRegression:
    def __init__(self, number_of_features, learning_rate=0.001, number_of_iterations=100):
        self.learning_rate = learning_rate
        self.number_of_iterations = number_of_iterations
        self.beta = np.random.randn(number_of_features, 1)
        self.b = 0
    
    def _map_binario(self, pred):
        # Funcion para realizar el mapeo a 0 y 1 de los valores de la regresion
        if pred >= 0.5:
            return 1
        else:
            return 0
        
    def train(self, X, y):
        # Se cambia la serie de pandas a un array de numpy
        y = y.to_numpy()

        # Se realiza por cada iteracion
        for iteration in range(self.number_of_iterations):
            #Se calcula las predicciones con la funcion sigmoide
            Z=np.dot(self.beta.T,X.T) + self.b
            y_pred = sigmoid(Z)

            #Se calcula las gradientes y B0
            gradients=1/len(X)*np.dot(X.T,(y_pred-y).T)
            constant=1/len(X)*np.sum(y_pred-y)

            #Se reajusta los valores
            self.beta = self.beta - self.learning_rate * gradients
            self.b = self.b - self.learning_rate * constant
        pass
    
    def predict(self, value):
        #Se hace las predicciones con la funcion sigmoide
        Z=np.dot(self.beta.T,value.T) + self.b
        y_pred = sigmoid(Z)

        #Se hace un mapeo donde los valores >= 0.5 cambian a 1 y el resto cambian a 0
        res = np.array([self._map_binario(pred) for pred in y_pred[0]])
        
        return res
        pass

# Ejemplo de uso de la clase para 3 features
log_reg = LogisticRegression(3)
print(log_reg.beta)

[[-0.45479995]
 [ 1.37636698]
 [ 0.21892173]]


## Parte 2 (1 pto): entrenando el modelo, computando el log loss

En esta parte tendrás que hacer un clasificador de gustos de helados tal como lo hicimos para regresión logística. 

In [123]:
import pandas as pd
from sklearn.metrics import log_loss
helados = pd.read_csv('Ice_cream.csv')
helados.info()

X = helados[['female','video','puzzle']]
y = helados['ice_cream']
# Entrena el modelo acá, debería ser como la siguiente línea
log_reg.train(X, y)
# Ahora usa el predict
# predicciones = log_reg.predict(X, y)
predicciones = log_reg.predict(X)
print(log_reg.beta)
#y obtiene el log loss
print("\nVALOR LOG LOSS:")
log_loss(y, predicciones)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  200 non-null    int64
 1   id          200 non-null    int64
 2   female      200 non-null    int64
 3   ice_cream   200 non-null    int64
 4   video       200 non-null    int64
 5   puzzle      200 non-null    int64
dtypes: int64(6)
memory usage: 9.5 KB
[[-0.46200683]
 [ 0.45202139]
 [-0.44190163]]

VALOR LOG LOSS:


15.88802904504216

## Parte 3 (1.5 ptos) mini-batch gradient descent

Otra forma de implementar el _gradient descent_ es mediante batches, o pedazos. La idea es la siguiente: 
- Se selecciona un número B de pedazos a usar y un número E de épocas
- Para cada época: 
    - Dividir el set de entrenamiento en B pedazos 
    - Para cada pedazo: 
        - calcular el gradiente usando la fórmula, pero solo para los datos de ese pedazo. 
        - realizar una iteración del gradient descent y actualizar los coeficientes
        
En esta última parte, modifica tu clase LogisticRegression para que el train sea hecho con mini-bath gradient descent. Las épocas y los batches deben ser pasados como parámetros. 

In [124]:
# Tienes que programar la parte 3 aquí
import numpy as np

# La función sigmoide
def sigmoid(x):    
    output = 1 / (1 + np.exp(-x))
    return output

class LogisticRegressionBatches:
    def __init__(self, number_of_features,n_batches,n_epochs, learning_rate=0.001):
        self.learning_rate = learning_rate
        self.n_batches = n_batches
        self.n_epochs = n_epochs
        self.beta = np.random.randn(number_of_features, 1)
        self.b = 0

    def _map_binario(self, pred):
        # Funcion para realizar el mapeo a 0 y 1 de los valores de la regresion
        if pred >= 0.5:
            return 1
        else:
            return 0
        
    def train(self, X, y):
        X = X.to_numpy()
        y = y.to_numpy()

        # Se realiza por cada epoca
        for iteration in range(self.n_epochs):

            # Metodo de shuffle obtenido de https://towardsdatascience.com/an-introduction-to-gradient-descent-c9cca5739307
            shuffle = list(np.random.permutation(len(X)))
            sX = X[shuffle,:]
            sy = y[shuffle]

            batchesx = np.array_split(X,self.n_batches)
            batchesy = np.array_split(y,self.n_batches)

            for batch in range(self.n_batches):
                segmento = batchesx[batch]
                resultados = batchesy[batch]

                #Se calcula las predicciones con la funcion sigmoide
                Z=np.dot(self.beta.T,segmento.T) + self.b
                y_pred = sigmoid(Z)

                #Se calcula las gradientes y B0
                gradients=1/len(segmento)*np.dot(segmento.T,(y_pred-resultados).T)
                constant=1/len(segmento)*np.sum(y_pred-resultados)

                #Se reajusta los valores
                self.beta = self.beta - self.learning_rate * gradients
                self.b = self.b - self.learning_rate * constant


        pass
    
    def predict(self, value):
        Z=np.dot(self.beta.T,value.T) + self.b
        y_pred = sigmoid(Z)

        #Se hace un mapeo donde los valores >= 0.5 cambian a 1 y el resto cambian a 0
        res = np.array([self._map_binario(pred) for pred in y_pred[0]])
        
        return res
        pass

## Parte 4 (0.5 pts): número de batches y épocas?

Lo último que debes hacer es encontrar un número de batches y épocas de forma que el modelo que entrenas con esos batches y épocas entregue un log-loss similar al que obtuviste con el método de gradient descent en la parte 1-2. 

In [142]:
helados = pd.read_csv('Ice_cream.csv')
helados.info()

mb_log_reg = LogisticRegressionBatches(3,5,100)


X = helados[['female','video','puzzle']]
y = helados['ice_cream']
# Entrena el modelo acá, debería ser como la siguiente línea
mb_log_reg.train(X, y)

# Ahora usa el predict
# predicciones = log_reg.predict(X, y)
predicciones = mb_log_reg.predict(X)
print(mb_log_reg.beta)
#y obtiene el log loss
print("\nVALOR LOG LOSS:")
log_loss(y, predicciones)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  200 non-null    int64
 1   id          200 non-null    int64
 2   female      200 non-null    int64
 3   ice_cream   200 non-null    int64
 4   video       200 non-null    int64
 5   puzzle      200 non-null    int64
dtypes: int64(6)
memory usage: 9.5 KB
[[ 0.18692105]
 [ 0.21974854]
 [-0.2169118 ]]

VALOR LOG LOSS:


15.888037041016462

## Detalles académicos

La entrega de este control debe ser un solo archivo **Jupyter Notebook**. **La fecha de entrega es hasta el viernes 10 de Septiembre, hasta las 20:00 pm**. La nota se calcula como el número de puntos + un punto base. El archivo se entrega en un cuestionario en siding. 